In [1]:
def create_embeded_matrix(dataAddress='./../Data/eng.train',frq = 2,debug=0):
    if( debug == 1 ):
        print("---------inside create_embeded_matrix function---------\n")
        
    assert(debug==0 or debug ==1)
    
    with open(dataAddress,'r') as data:
        line = data.readlines()
    
    if( debug == 1 ):
        print("Total data : %d"%(len(line)))
    
    isO   = {}
    all_X = []
    all_Y = []
    cnt = 0 ; 
    for i in line:
        temp = i.split()
        l = len(temp)     
        if( l != 4 ):
            continue
        if( temp[3] == "O" ):
            cnt = cnt+1 ; 
        all_X.append(temp[0]) ;
        if( temp[0] not in isO ):
            isO[ temp[0] ]  = 0 
        if( temp[3] != "O" ):
            isO[ temp[0] ]  = 1 
    
    if( debug == 1):
        print("Total Number of **Other** NE =",cnt)
        init_reduce = len(line)-len(all_X) 
        print("Initial reduction : %d"%(init_reduce))
        print("Current total : %d"%len(all_X))
        
    
    
    frqCnt=dict()
    for i in all_X :
        if( i not in frqCnt ):
            frqCnt[i] = 1
        else: 
            frqCnt[i] = frqCnt[i]+1
            
    if( debug == 1):
        print("Size of the dictionary(Unique Words) : %d"%len(frqCnt))
        p = set(all_X) ;
        assert( len(p) == len(frqCnt) )
    
    
    shrinked_X = []
    for k,v in frqCnt.items():
        if( (v >= frq) and (isO[k] == 1) ):
            shrinked_X.append(k) 
    shrinked_X.append("*?*other")
    
    assert(len(shrinked_X) == len(set(shrinked_X)))
    
    em_hash = dict()
    idx = 0 
    for i in shrinked_X:
        if i not in em_hash:
            em_hash[i] = idx
        idx = idx + 1
    
    if( debug == 1):
        print("Total item after shrinking by frequency-{0} : {1}".format(frq, len(shrinked_X)))
    
    if( debug == 1 ):
        print("\n---------end of create_embeded_matrix function---------")
    
    return (shrinked_X,em_hash)

In [2]:
def output_map(debug=0):
    """
        Category defination
        O = others
        I/B = 2
        LOC/MISC/ORG/PER = 4
        Total number of Combination = 1+2*4 = 9
    """
    if( debug == 1 ):
        print("---------inside output_map function---------\n")
    assert(debug ==0 or debug == 1)
    
    IB=["I","B"]
    ST=["LOC","MISC","ORG","PER"]
    output_type = []
    for i in IB:
        for j in ST:
            output_type.append(i+"-"+j)
    output_type.append("O")
    
    ret = dict()
    idx = 0 
    for i in output_type:
        ret[i] = idx
        if( debug == 1 ):
             print(i+"'s index number is %d"%idx )
        idx = idx+1
    if( debug == 1 ):
        print("\n---------end of inside output_map function---------")
    return ret

In [3]:
import numpy as np
def parse_input(dataAddress='./../Data/eng.train',debug=0):
    if( debug == 1 ):
        print("---------inside parse function---------\n")
        
    assert(debug==0 or debug ==1)
    
    with open(dataAddress,'r') as data:
        line = data.readlines()
    
    if( debug == 1 ):
        print("Total data : %d"%(len(line)))
    

    all_X = []
    all_Y = []
    for i in line:
        temp = i.split()
        l = len(temp) 
        if( l != 4 ):
            continue
        #idx = output_hash[temp[3]]
        #temp_one_hot_Y = np.zeros(len(output_hash));
        #temp_one_hot_Y[idx] = 1 ;
        
        all_X.append(temp[0]) ;
        all_Y.append(temp[3]) ;
    
 
    if( debug == 1):
        init_reduce = len(line)-len(all_X) 
        print("Initial reduction : %d"%(init_reduce))
        print("Current total : %d"%len(all_X))
        assert(len(all_X)==len(all_Y))
    
    if( debug == 1 ):
        print("\n---------end of parse function---------")
    return all_X,all_Y

# information from datasets

## eng.testa
-----------
+ I-LOC
+ B-MISC
+ O
+ I-MISC
+ I-PER
+ I-ORG

## eng.testb
+ I-LOC
+ B-MISC
+ B-LOC
+ O
+ I-MISC
+ I-PER
+ I-ORG

## eng.train
+ I-LOC
+ I-MISC
+ I-PER
+ I-ORG
+ B-LOC
+ B-MISC
+ B-ORG
+ O

## temporary archived code
"""
import math
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

RND_SEED = 100
tf.set_random_seed(RND_SEED)


    Embeded Matrix
    EM = create_embeded_matrix(debug=1)
    EM_X = EM[0] 
    EM_hash = EM[1]
    take all the inputs from the eng.train. 
    output is represented in one hot encoding format
    train_X, train_y = parse_input(debug=1)
    number_of_softmax_layer = len(output_map())
    

    #design model
    size_of_embeded_row = 100
    for i in range(total_no_of_word):
        W[i] = tf.Variable(tf.zeros([size_of_embeded_row = 100,number_of_softmax_layer]))
        x[i] = tf.placeholder(tf.float32, [1, 100])
        y += W[i]*x[i] ;

    vocabulary_size = len(EM_X)
    embedding_size = 100 
    word_embeddings = tf.get_variable("word_embeddings",[vocabulary_size, embedding_size])
    
    batch_size = len(EM_X)
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 9])
    
    embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, train_inputs)
    print(embedded_word_ids.shape)
    
    nce_weights = tf.Variable(
                      tf.truncated_normal([vocabulary_size, embedding_size],
                      stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    num_sampled = batch_size;
    init_loss = tf.reduce_mean(
                 tf.nn.nce_loss(weights=nce_weights,
                 biases=nce_biases,
                 labels=train_labels,
                 inputs=embedded_word_ids,
                 num_sampled=9,
                 num_classes=vocabulary_size))
    loss = tf.nn.softmax(init_loss)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=.5).minimize(loss)
    
    feed_dict = {train_inputs:range(len(train_X)) ,def main(): train_labels: train_Y}
    _, cur_loss = session.run([optimizer, loss], feed_dict=feed_dict)
"""
    
    
    

In [4]:
def get_batch_input(batch_no,train_X,train_Y,batch_size=100):
    row_st = batch_no*batch_size
    if( row_st > len(train_X) ):
        return [],[]
    return train_X[row_st:min(row_st+batch_size,len(train_X)+1)],train_Y[row_st:min(row_st+batch_size,len(train_Y)+1)]

In [5]:
def one_hot_embedding(vector,one_hot_hash):
    retVec = []
    for idx,val in enumerate(vector):
        tmp_one_hot = np.zeros(len(one_hot_hash))
        if val in one_hot_hash:
            tmp_one_hot[ one_hot_hash[val] ] = 1 
        else:
            tmp_one_hot[ -1 ] = 1 
        retVec.append(tmp_one_hot)
    return retVec

# With SGD/RMSProp/ADAM Training algorithm


In [6]:
import math
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import sys

RND_SEED = 100
tf.set_random_seed(RND_SEED)

def main():
    
    """
        Hyper parameter list
        
        + EMBEDDING_DIM
               
    """
    
    #Embeded Matrix
    EM = create_embeded_matrix(debug=1)
    EM_X = EM[0] 
    EM_hash = EM[1]
    #print(EM_hash['EU'])
    
    #take all the inputs from the eng.train. 
    #output is represented in one hot encoding format
    train_X, train_Y = parse_input(debug=1)
    
    train_X = np.asarray(train_X)
    train_Y = np.asarray(train_Y)
    
    #print(train_X[1:10])
    #print(train_Y[1:10])
    
    Y_hash = output_map()
    output_size = len(Y_hash)
    
    #print(train_X.shape,train_Y.shape)

    vocabulary_size = len(EM_X)
    x = tf.placeholder(tf.float32,shape=[None,vocabulary_size])
    y_goal = tf.placeholder(tf.float32, shape=[None,output_size])
    
    EMBEDDING_DIM = 100

    W1 = tf.Variable(tf.random_normal([vocabulary_size,EMBEDDING_DIM]))
    b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM]))
    hidden_representation = tf.add(tf.matmul(x,W1),b1)

    W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM,output_size]))
    b2 = tf.Variable(tf.random_normal([output_size]))
    prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_representation,W2),b2))
    
    cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_goal*tf.log(prediction),reduction_indices=[1]))
    
    #train_stepSGD = tf.train.GradientDescentOptimizer(.00001).minimize(cross_entropy_loss)
    train_stepRMS = tf.train.RMSPropOptimizer(.0001).minimize(cross_entropy_loss)
    #train_stepADAM = tf.train.AdamOptimizer(learning_rate=.0001).minimize(cross_entropy_loss)

    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    epoch = 500
    
    
    print("\nEntering training Session")
    print("-------------------------")
    for itr in range(epoch):
        batch_no = 0  
        a = -1
        b = -1 
        while( 1 ):
            curr_X, curr_Y = get_batch_input(batch_no,train_X,train_Y,batch_size=100)
            if( len(curr_X) == 0 ):
                break
            curr_X = one_hot_embedding(curr_X,EM_hash)
            curr_Y = one_hot_embedding(curr_Y,Y_hash)
            curr_X = np.asarray(curr_X)
            curr_Y = np.asarray(curr_Y)
            #a, b = sess.run([train_stepSGD, cross_entropy_loss],feed_dict={x:curr_X,y_goal:curr_Y})
            a, b = sess.run([train_stepRMS, cross_entropy_loss],feed_dict={x:curr_X,y_goal:curr_Y})
            #a, b = sess.run([train_stepADAM, cross_entropy_loss],feed_dict={x:curr_X,y_goal:curr_Y})
            batch_no = batch_no + 1 
            #print(b)
        #print("{0} epoch completed. current SGD loss {1}".format(itr+1,b))
        print("{0} epoch completed. current RMS loss {1}".format(itr+1,b))
        #print("{0} epoch completed. current ADAM loss {1}".format(itr+1,b))


In [7]:
import time
#with open('./log.txt', 'w') as f:    
    #sys.stdout = f
print("Experiment taken on : ",time.strftime("%c"))
start_time = time.time()
main()
end_time = (time.time() - start_time)
print("--- %s seconds ---" % end_time)

Experiment taken on :  Tue Oct 17 17:20:41 2017
---------inside create_embeded_matrix function---------

Total data : 219553
Total Number of **Other** NE = 170524
Initial reduction : 14986
Current total : 204567
Size of the dictionary(Unique Words) : 23624
Total item after shrinking by frequency-2 : 4575

---------end of create_embeded_matrix function---------
---------inside parse function---------

Total data : 219553
Initial reduction : 14986
Current total : 204567

---------end of parse function---------

Entering training Session
-------------------------
1 epoch completed. current RMS loss 4.713581085205078
2 epoch completed. current RMS loss 3.8941619396209717
3 epoch completed. current RMS loss 3.4585413932800293
4 epoch completed. current RMS loss 3.0551810264587402
5 epoch completed. current RMS loss 2.645879030227661
6 epoch completed. current RMS loss 2.267488718032837
7 epoch completed. current RMS loss 1.968588948249817
8 epoch completed. current RMS loss 1.79431498050689

136 epoch completed. current RMS loss 0.2620222270488739
137 epoch completed. current RMS loss 0.26234304904937744
138 epoch completed. current RMS loss 0.2626311480998993
139 epoch completed. current RMS loss 0.26287832856178284
140 epoch completed. current RMS loss 0.2630859911441803
141 epoch completed. current RMS loss 0.2632542848587036
142 epoch completed. current RMS loss 0.26338690519332886
143 epoch completed. current RMS loss 0.2634749412536621
144 epoch completed. current RMS loss 0.263528436422348
145 epoch completed. current RMS loss 0.2635319232940674
146 epoch completed. current RMS loss 0.2634921073913574
147 epoch completed. current RMS loss 0.2634023427963257
148 epoch completed. current RMS loss 0.26327642798423767
149 epoch completed. current RMS loss 0.2631160020828247
150 epoch completed. current RMS loss 0.2629314959049225
151 epoch completed. current RMS loss 0.2627306580543518
152 epoch completed. current RMS loss 0.2625216245651245
153 epoch completed. current

279 epoch completed. current RMS loss 0.2333282083272934
280 epoch completed. current RMS loss 0.2330635040998459
281 epoch completed. current RMS loss 0.23279541730880737
282 epoch completed. current RMS loss 0.23251773416996002
283 epoch completed. current RMS loss 0.23223187029361725
284 epoch completed. current RMS loss 0.23194007575511932
285 epoch completed. current RMS loss 0.23165152966976166
286 epoch completed. current RMS loss 0.23135510087013245
287 epoch completed. current RMS loss 0.23105226457118988
288 epoch completed. current RMS loss 0.23075580596923828
289 epoch completed. current RMS loss 0.23044466972351074
290 epoch completed. current RMS loss 0.23012971878051758
291 epoch completed. current RMS loss 0.22982074320316315
292 epoch completed. current RMS loss 0.22949102520942688
293 epoch completed. current RMS loss 0.22916792333126068
294 epoch completed. current RMS loss 0.22883649170398712
295 epoch completed. current RMS loss 0.22850261628627777
296 epoch comple

421 epoch completed. current RMS loss 0.2076127678155899
422 epoch completed. current RMS loss 0.20735858380794525
423 epoch completed. current RMS loss 0.2071167230606079
424 epoch completed. current RMS loss 0.20687778294086456
425 epoch completed. current RMS loss 0.20664405822753906
426 epoch completed. current RMS loss 0.20641128718852997
427 epoch completed. current RMS loss 0.20617720484733582
428 epoch completed. current RMS loss 0.2059541791677475
429 epoch completed. current RMS loss 0.20571820437908173
430 epoch completed. current RMS loss 0.20547887682914734
431 epoch completed. current RMS loss 0.2052338421344757
432 epoch completed. current RMS loss 0.20498254895210266
433 epoch completed. current RMS loss 0.20472753047943115
434 epoch completed. current RMS loss 0.20446759462356567
435 epoch completed. current RMS loss 0.20419743657112122
436 epoch completed. current RMS loss 0.2039305865764618
437 epoch completed. current RMS loss 0.2036735713481903
438 epoch completed.